In [2]:
import re
import contractions
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from string import digits

import numpy as np
import pandas as pd
import os
# from https://jon-dagdagan.medium.com/fake-news-detection-pre-processing-text-d9648a2854e5

In [3]:
import json
import newspaper
import string

In [5]:
articles = pd.read_csv('texts_and_labels_notcleaned.tsv', sep='\t')

In [9]:
articles

,Unnamed: 0,text,label,bias
0,0,Matt O'Brien\n\nAssociated Press\n\nYouTube is...,elections-2020,center
1,1,"FRISCO, Texas — The increasingly bitter disput...",sport,left
2,2,Speaking to the country for the first time fro...,immigration,left
3,3,A professor who teaches climate change classes...,environment,right
4,4,The left has a thing for taking babies hostage...,abortion,right
...,...,...,...,...
1005,1597,Colorado students walked out of an event bille...,gun-control,right
1006,1600,Mainstream media outlets continue to print fal...,white-nationalism,right
1007,1602,During the months-long Republican effort to te...,international-politics-and-world-news,left
1008,1608,WASHINGTON – New Mexico Rep. Debra Haaland got...,gender,center


In [3]:
# file = 'News_Category_Dataset_v3.json'

In [4]:
# article_info = []
# for line in open(file, 'r'):
#     article_info.append(json.loads(line))

In [5]:
# article_info[0]

{'link': 'https://www.huffpost.com/entry/covid-boosters-uptake-us_n_632d719ee4b087fae6feaac9',
 'headline': 'Over 4 Million Americans Roll Up Sleeves For Omicron-Targeted COVID Boosters',
 'category': 'U.S. NEWS',
 'short_description': 'Health experts said it is too early to predict whether demand would match up with the 171 million doses of the new boosters the U.S. ordered for the fall.',
 'authors': 'Carla K. Johnson, AP',
 'date': '2022-09-23'}

In [6]:
# article_info_short = article_info[:50000]

In [ ]:
# categories = []
# fails = []
# for article in article_info_short:
#     url = article['link']
#     try:
#         url_i = newspaper.Article(url="%s" % (url), language='en')
#         url_i.download()
#         url_i.parse()
#         article_texts.append(url_i.text)
#         categories.append(article['category'])
#     except:
#         fails.append(url)

In [ ]:
# category_articles = pd.DataFrame(
#     {'article_text': article_texts,
#      'category': categories,
#     })

In [6]:
# category_articles.to_csv('unprocessed_50k_HuffpostClassification.tsv', sep="\t")

In [7]:
tokenizer = RegexpTokenizer(r'\w+')
nltk.download('words') #download list of english words
nltk.download('stopwords') #download list of stopwords
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
stopWords = stopwords.words('english')
englishWords = set(nltk.corpus.words.words())

new_stopwords = ["associate press", "reuters"]
stopWords.extend(new_stopwords)


from nltk.corpus import wordnet
lemmatizer = WordNetLemmatizer()

def nltkToWordnet(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:                    
        return None

def lemmatize(tokens):
    pos_tags = nltk.pos_tag(tokens)    
    res_words = []
    for word, tag in pos_tags:
        tag = nltkToWordnet(tag)    
        if tag is None:                        
            res_words.append(word)
        else:
            res_words.append(lemmatizer.lemmatize(word, tag))
    return res_words

def remove_stopWords(tokens):
    return [w for w in tokens if (w in englishWords and w not in stopWords)]

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Jessica\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Jessica\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Jessica\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Jessica\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
# # directory
# text_dir = os.path.join(os.getcwd(), 'labeled')
# # change depending on your text source
# texts = np.load(os.path.join(text_dir, 'docs_text.npy'))

# # article is the article
# article = texts[60]

In [31]:
def process_text(text):
    # Remove some author/publisher info
    if len(text.split("(AP) —")) > 1:
        text = "(AP) —".join(text.split("(AP) —")[1:])
    if len(text.split("___")) > 1:
        text = "___".join(text.split("___")[:-1])
    if len(text.split("Associated Press writers")) > 1:
        text = "Associated Press writers".join(text.split("Associated Press writers")[:-1])
    if len(text.split("(Reuters)")) > 1:
        text = "(Reuters)".join(text.split("(Reuters)")[1:])
    if len(text.split("Reporting by")) > 1:
        text = "Reporting by".join(text.split("Reporting by")[:-1])
    # remove all text in quotes
    for x in re.finditer('“(.*?)”', text):
        text = re.sub(x.group(), '', text)
    # remove digits
    text = text.translate(str.maketrans('', '', digits))
    # Lowercase and remove URLs
    text = re.sub(r'http\S+', '', text.lower())
    # Remove "advertisement"
    text = re.sub('advertisement', '', text.lower())
    # Split up contractions
    text = ' '.join([contractions.fix(word) for word in text.split()])
    # Tokenize text
    tokens = tokenizer.tokenize(text)
    # Lemmatize tokens
    tokens = lemmatize(tokens)
    # Remove stopwords
    tokens = remove_stopWords(tokens)
    # Print results
    result = ' '.join(tokens)
    
    return result

In [32]:
articles['text'][315]

nan

In [33]:
processed_texts = []
count = 0
for article in articles['text']:
    try:
        result = process_text(article)
        processed_texts.append(result)
    except:
        processed_texts.append("error")
    # print(count)
    count+=1

In [ ]:
# data['column_name'].value_counts()[value]

In [34]:
processed_texts.count('error')

5

In [36]:
articles['text'] = processed_texts

In [38]:
articles.to_csv('texts_and_labels_cleaned.tsv', sep='\t')

In [ ]:
category_articles_processed = pd.DataFrame(
    {'article_text': processed_texts,
     'category': categories,
    })

category_articles_processed.to_csv('processed_50k_HuffpostClassification.tsv', sep="\t")

In [ ]:
# text = article

# # Remove some author/publisher info
# if len(text.split("(AP) —")) > 1:
#     text = "(AP) —".join(text.split("(AP) —")[1:])
# if len(text.split("___")) > 1:
#     text = "___".join(text.split("___")[:-1])
# if len(text.split("Associated Press writers")) > 1:
#     text = "Associated Press writers".join(text.split("Associated Press writers")[:-1])
# if len(text.split("(Reuters)")) > 1:
#     text = "(Reuters)".join(text.split("(Reuters)")[1:])
# if len(text.split("Reporting by")) > 1:
#     text = "Reporting by".join(text.split("Reporting by")[:-1])
 

# # remove all text in quotes
# for x in re.finditer('“(.*?)”', text):
#     text = re.sub(x.group(), '', text)
    
# # remove digits
# text = text.translate(str.maketrans('', '', digits))

# # Lowercase and remove URLs
# text = re.sub(r'http\S+', '', text.lower())
# # Remove "advertisement"
# text = re.sub('advertisement', '', text.lower())

# # Split up contractions
# text = ' '.join([contractions.fix(word) for word in text.split()])

# # Tokenize text
# tokens = tokenizer.tokenize(text)

# # Lemmatize tokens
# tokens = lemmatize(tokens)

# # Remove stopwords
# tokens = remove_stopWords(tokens)

# # Print results
# # result = ' '.join(tokens)